In [165]:
import pandas as pd

In [166]:
orders = pd.read_csv('data/orders.csv', index_col='payment_id')
orders.head(10)

,user_id,created_at,payment_sum
payment_id,,,
2,2,2019-12-09 16:57:19.000,42.26
4,4,2019-12-10 03:49:35.000,18.25
6,6,2019-12-12 00:19:08.000,10.52
7,7,2019-12-13 15:15:30.000,14.02
8,8,2019-12-14 00:02:06.000,10.78
10,10,2019-12-17 04:57:55.000,27.68
12,12,2019-12-18 23:50:11.000,12.71
15,15,2019-12-20 18:28:57.000,19.06
16,16,2019-12-21 19:10:52.000,17.82


In [167]:
# задача: посчитать окупаемость маркетинга за последние 12 месяцев

# тратим деньги 
# привлекаем пользователей
# пользователи что-то покупают

# когорты от даты регистрации 
# посчитаем 
# — выручку и LTV 
# - рекламные расходы
# - окупаемость маркетинга

In [168]:
# Алгоритм
# 0. Формилируем задачу
# 1. Определяем когорту (событие и временной промежуток) - дата регистрации по месяцам
# 2. Выделяем целевые метрики - выручка, LTV, ROAS
# 3. Выбираем подходящий формат отчёта - возрастной 

In [187]:
ads = pd.read_csv('data/ads.csv')
ads['dt'] = pd.to_datetime(ads['dt'])
ads['month'] = ads['dt'] .dt.to_period('M')
# добавили данные по рекламе и трансформировали даты

monthly_ads = ads.groupby('month').agg(ad_spend = ('ad_spend', 'sum'))

In [169]:
users = pd.read_csv('data/users.csv', index_col='id')
users['reg_date'] = pd.to_datetime(users['reg_date'])
users['reg_month'] = users['reg_date'] .dt.to_period('M')
users[['reg_date', 'reg_month']]

,reg_date,reg_month
id,,
1124,2020-06-28,2020-06
1766,2020-09-20,2020-09
4132,2021-09-01,2021-09
3662,2021-06-23,2021-06
4804,2021-12-15,2021-12
...,...,...
3895,2021-07-27,2021-07
4235,2021-09-21,2021-09
1776,2020-09-22,2020-09


In [170]:
cohort_size = users.groupby('reg_month').agg(n_users = ('email', 'count')).reset_index()
# cчитаем количество пользователей в когорте

In [171]:
# загрузили данные по выручке
orders = pd.read_csv('data/orders.csv', index_col='payment_id')
orders['created_at'] = pd.to_datetime(orders['created_at'])

# находим первый заказ каждого пользователя через группировку по минимальной дате
orders = (
    orders
    .groupby('user_id')
    .agg(first_payment_at = ('created_at', 'min'))
    .merge(orders, how='inner', left_index=True, right_on='user_id')
) 
orders['payment_month'] = orders['created_at'].dt.to_period('M')
orders['first_payment_month'] = orders['first_payment_at'].dt.to_period('M')


orders = orders.merge(
    users[['reg_date', 'reg_month']], 
    how='inner',
    left_on='user_id', 
    right_index=True)

orders.head(5)


orders['cohort_age_days'] = (orders['created_at'] - orders['reg_date']).dt.days
# # получили когорты по дням с момента создания 

orders['cohort_age_months'] = orders['cohort_age_days'] // 30
# получили когорты по месяцам, где // - делит без остатка


In [172]:
cohort_size.tail(10)

,reg_month,n_users
26,2022-02,157
27,2022-03,162
28,2022-04,170
29,2022-05,233
30,2022-06,234
31,2022-07,276
32,2022-08,235
33,2022-09,233
34,2022-10,213
35,2022-11,165


In [206]:
# делаем когортный отчёт через groupby
cohorts = (
    orders
    .groupby(['reg_month', 'cohort_age_months', 'payment_month'])
    .agg(revenue = ('payment_sum', 'sum'))
    .reset_index()
)

cohorts = cohorts.merge(cohort_size, how='outer', on='reg_month')
#обьединили когорты и количество пользователей по индексу

cohorts['rev_per_user'] = cohorts['revenue'] / cohorts['n_users']
# посчитали выручку на 1 юзера с когорты

cohorts['ltv'] = cohorts.groupby('reg_month')['rev_per_user'].expanding().sum().values

# посчитали ЛТВ, суммировав ревенью на юзера

cohorts = (
    cohorts
    .merge(monthly_ads, 
           how='outer', 
           left_on='reg_month',
           right_on='month')
)
# большая таблица с ревенью на юзера 

# roas 
cohorts['cac'] = cohorts['ad_spend'] / cohorts['n_users']
cohorts['roas'] = cohorts['ltv'] / cohorts['cac']

In [207]:
cohorts.head()

,reg_month,cohort_age_months,payment_month,revenue,n_users,rev_per_user,ltv,ad_spend,cac,roas
0,NaT,NaN,NaT,NaN,NaN,NaN,NaN,43.4,NaN,NaN
1,2019-12,0.0,2019-12,479.20,22.0,21.781818,21.781818,3052.6,138.754545,0.156981
2,2019-12,0.0,2020-01,54.18,22.0,2.462727,24.244545,3052.6,138.754545,0.174730
3,2019-12,1.0,2020-01,47.51,22.0,2.159545,26.404091,3052.6,138.754545,0.190294
4,2019-12,1.0,2020-02,31.55,22.0,1.434091,27.838182,3052.6,138.754545,0.200629


In [210]:
pd.pivot(
    cohorts,
    index='reg_month',
    columns='cohort_age_months',
    values='roas'
).fillna()

ValueError: Index contains duplicate entries, cannot reshape

In [189]:
monthly_ads

,ad_spend
month,
2019-11,43.400000
2019-12,3052.600000
2020-01,10912.514286
2020-02,13781.914286
2020-03,21480.142857
2020-04,23378.228571
2020-05,26373.000000
2020-06,29688.828571
2020-07,44067.614286


TypeError: Can only merge Series or DataFrame objects, a <class 'method'> was passed

In [123]:
cohort_size

,reg_month,n_users
0,2019-12,22
1,2020-01,107
2,2020-02,113
3,2020-03,110
4,2020-04,89
5,2020-05,114
6,2020-06,121
7,2020-07,159
8,2020-08,157
9,2020-09,140


In [124]:
#через пивоты
# # cohorts = (
#     orders
#     .pivot_table(
#         index='first_payment_month', 
#         columns='cohort_age_month',
#         values='payment_sum',
#         aggfunc='sum'
#     )
# )

cohorts

,reg_month,cohort_age_month,payment_month,revenue,n_users,rev_per_user
0,2019-12,0,2019-12,479.20,22,21.781818
1,2019-12,0,2020-01,54.18,22,2.462727
2,2019-12,1,2020-01,47.51,22,2.159545
3,2019-12,1,2020-02,31.55,22,1.434091
4,2019-12,2,2020-03,46.81,22,2.127727
...,...,...,...,...,...,...
1284,2022-09,2,2022-11,657.94,233,2.823777
1285,2022-10,0,2022-10,6904.14,213,32.413803
1286,2022-10,0,2022-11,938.17,213,4.404554
1287,2022-10,1,2022-11,908.57,213,4.265587


In [125]:
[cohorts.at[m,m] for m in cohorts.columns][0:5] 
# вытаскиваем значения первых оплат
# [np.float64(479.2),
# np.float64(3003.31),

cohorts.sum(axis='rows').head(5)
#cчитаем суммы по месяцам
# payment_month
# 2019-12     479.20
# 2020-01    3105.00

[cohorts.at[m,m] for m in cohorts.columns] / cohorts.sum(axis='rows')
# 30% выручки приходит от новых пользвоателей. выше мы сумму от новых пользователей 
# делим на общую сумму за месяц и получаем отношение
# 70% выручки приходит от старых. 

KeyError: 'reg_month'